# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import numpy as np

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_file = "output_data/cities.csv"
city_data_df = pd.read_csv(city_data_file)


In [3]:
# Display dataset
city_data_df.head()

,City_ID,City_ID.1,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points("Lng", "Lat", geo = True, frame_height= 500, frame_width= 700, size = "Humidity", color = "City")

# Display the map
map_plot_1

:Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# Filter vacation with zero clodiness
narrow_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]


# Drop any rows with null values
narrow_city_df = narrow_city_df.dropna(how = "any")

# Display sample data
narrow_city_df.head(20)

,City_ID,City_ID.1,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,45,kapaa,22.0752,-159.3190,22.99,84,0,3.60,US,1666108257
51,51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
63,63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
81,81,81,makakilo city,21.3469,-158.0858,21.66,81,0,2.57,US,1666108282
152,152,152,kahului,20.8947,-156.4700,23.80,60,0,3.09,US,1666108246
197,197,197,gat,31.6100,34.7642,24.38,100,0,3.69,IL,1666108356
211,211,211,laguna,38.4210,-121.4238,21.67,79,0,2.06,US,1666108364
240,240,240,tikaitnagar,26.9500,81.5833,23.56,59,0,0.35,IN,1666108378
265,265,265,san quintin,30.4833,-115.9500,21.20,74,0,1.37,MX,1666108394
340,340,340,santa rosalia,27.3167,-112.2833,24.62,56,0,0.74,MX,1666108436


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_city_df[["City", "Country", "Lat", "Lng", "Humidity"]]
hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ''

# Display sample data
hotel_df.head(20)

C:\Users\pato2\AppData\Local\Temp\ipykernel_21196\4212199214.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]= ''


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,
51,hilo,US,19.7297,-155.0900,83,
63,banda,IN,25.4833,80.3333,52,
81,makakilo city,US,21.3469,-158.0858,81,
152,kahului,US,20.8947,-156.4700,60,
197,gat,IL,31.6100,34.7642,100,
211,laguna,US,38.4210,-121.4238,79,
240,tikaitnagar,IN,26.9500,81.5833,59,
265,san quintin,MX,30.4833,-115.9500,74,
340,santa rosalia,MX,27.3167,-112.2833,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
radius = 10000
params = {"categories": "accommodation.hotel", "apiKey": geoapify_key, "limit": 20}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_ df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = row["Lat"] 
    Lng = row["Lng"] 
    

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] =   f"proximity:{Lng},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    print(params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


{'categories': 'accommodation.hotel', 'apiKey': '1fb913e1135a426c89522c38f677f7c9', 'limit': 20, 'filter': 'circle:-159.319,22.0752,10000', 'bias': 'proximity:-159.319,22.0752'}
kapaa - nearest hotel: Pono Kai Resort
{'categories': 'accommodation.hotel', 'apiKey': '1fb913e1135a426c89522c38f677f7c9', 'limit': 20, 'filter': 'circle:-155.09,19.7297,10000', 'bias': 'proximity:-155.09,19.7297'}
hilo - nearest hotel: Dolphin Bay Hotel
{'categories': 'accommodation.hotel', 'apiKey': '1fb913e1135a426c89522c38f677f7c9', 'limit': 20, 'filter': 'circle:80.3333,25.4833,10000', 'bias': 'proximity:80.3333,25.4833'}
banda - nearest hotel: #acnindiafy21
{'categories': 'accommodation.hotel', 'apiKey': '1fb913e1135a426c89522c38f677f7c9', 'limit': 20, 'filter': 'circle:-158.0858,21.3469,10000', 'bias': 'proximity:-158.0858,21.3469'}
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
{'categories': 'accommodation.hotel', 'apiKey': '1fb913e1135a426c89522c38f677f7c9', 'limit': 20, 'filter'

,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,Jardines Hotel
340,santa rosalia,MX,27.3167,-112.2833,56,Sol y Mar


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot_2 = city_data_df.hvplot.points("Lng", "Lat", geo = True, frame_height= 500, frame_width= 700, size = "Humidity", color = "City", hover_cols=["Hotel Name", "Country"] )


# Display the map
map_plot_2

:Points   [Lng,Lat]   (City,Humidity,Country)